# Unsupervised Learning

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from collections import OrderedDict
L = LinearRegression(n_jobs=-1)
from functools import reduce
import sklearn
import sklearn.ensemble
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import chi2, f_regression, f_classif, mutual_info_classif,\
mutual_info_regression 
from sklearn.feature_selection import SelectKBest, SelectFromModel, VarianceThreshold
from mlxtend.frequent_patterns import apriori, association_rules

In [10]:
data = pd.read_csv('Preprocess.csv',parse_dates=['ClaimStartDt','ClaimEndDt','AdmissionDt','DOD','DischargeDt'])

## Diagnosis Code

In [3]:
def diag_freq(df):
    
    # Focus on ClmDiagnosisCode columns only
    columns = []
    for i in range(1,11):
        columns.append('ClmDiagnosisCode_'+str(i))
        
    new_df = df[columns]
    
    # List of diagnosis codes and their frequency in that column
    new_dct = []
    for y in new_df:
        new_dct.append(dict(df[y].value_counts(ascending=False,normalize=True)))
    
    # Filter diagnosis codes that make up more than 1% of the codes
    high_freq = []  
    for i in new_dct:    
        for key,value in i.items():
            if value > 0.01:
                high_freq.append(key)
            
    return high_freq

In [4]:
freq_diag = list(set(diag_freq(data)))

In [5]:
freq_diag

['V5869', 'None', 'V5861', '2724', '4011', '2449', '4019', '25000', '42731']

In [6]:
del freq_diag[1]

In [7]:
freq_diag

['V5869', 'V5861', '2724', '4011', '2449', '4019', '25000', '42731']

In [8]:
def newdf_diag_highfreq(df):
    columns = []
    for i in range(1,11):
        columns.append('ClmDiagnosisCode_'+str(i))
        
    freq_diag = ['V5869', 'V5861', '2724', '4011', '2449', '4019', '25000', '42731']    
    for col in freq_diag:
        df[str(col)+'_diagcode'] = np.where((df[columns].eq(col)).any(1, skipna=True), 1, 0)
        df['other_diagcode'] = np.where((df[columns].ne(col)).any(1, skipna=True), 1, 0)
        df['None_diagcode'] = np.where((df[columns].eq('None')).all(1,skipna=True), 1, 0)
    
    df = df.drop(df[columns],axis=1)
    return df

In [9]:
output = newdf_diag_highfreq(data)

In [10]:
pd.set_option('display.max_columns', None)
output

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,Inpatient,ClmProcedureCode,ClmProcedureCode_count,ClmDiagnosisCode_count,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,num_chronic,PotentialFraud,Age,V5869_diagcode,other_diagcode,None_diagcode,V5861_diagcode,2724_diagcode,4011_diagcode,2449_diagcode,4019_diagcode,25000_diagcode,42731_diagcode
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,None,None,2009-04-12 00:00:00,1,1068.0,2009-04-18 00:00:00,1.0,0.0,0.0,9.0,None,1,1,0.0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,7.0,1,66,0,1,0,0,1,0,0,1,0,0
1,BENE16973,CLM565430,2009-09-06,2009-09-06,PRV55912,50,PHY365867,PHY327147,None,None,0,0.0,None,0.0,0.0,0.0,9.0,None,1,1,0.0,39,310,12,12,1,1,1,0,0,0,0,1,0,0,0,24000,2136,450,200,4.0,1,77,0,1,0,0,0,0,0,0,0,0
2,BENE17521,CLM34721,2009-01-20,2009-02-01,PRV55912,19000,PHY349293,PHY370861,PHY363291,2009-01-20 00:00:00,1,1068.0,2009-02-01 00:00:00,1.0,1.0,2.0,9.0,None,2,1,0.0,39,230,12,12,1,1,1,0,1,0,0,1,0,0,0,19000,1068,100,20,5.0,1,95,0,1,0,0,0,0,0,0,0,0
3,BENE21718,CLM72336,2009-10-17,2009-11-04,PRV55912,17000,PHY334706,PHY334706,None,2009-10-17 00:00:00,1,1068.0,2009-11-04 00:00:00,1.0,1.0,1.0,9.0,None,1,1,0.0,39,600,12,12,0,0,0,0,0,0,0,1,1,0,0,17000,1068,1050,540,2.0,1,87,0,1,0,0,0,0,0,1,0,0
4,BENE22934,CLM73394,2009-10-25,2009-10-29,PRV55912,13000,PHY390614,PHY323689,PHY363291,2009-10-25 00:00:00,1,1068.0,2009-10-29 00:00:00,1.0,1.0,1.0,7.0,None,2,1,0.0,39,280,12,12,0,1,0,0,1,0,1,1,1,1,0,27000,2136,450,160,6.0,1,79,0,1,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,BENE154147,CLM394122,2009-06-02,2009-06-04,PRV54050,500,PHY317497,None,PHY317497,None,0,0.0,None,0.0,0.0,0.0,3.0,None,2,1,0.0,23,400,12,12,0,0,0,0,0,0,0,1,1,0,0,0,0,890,120,2.0,0,84,0,1,0,0,0,0,0,0,0,0
558207,BENE154687,CLM184358,2009-02-08,2009-02-08,PRV54302,3300,PHY376238,PHY376238,None,None,1,0.0,None,0.0,0.0,0.0,1.0,None,1,2,0.0,25,150,12,12,1,0,1,0,0,0,1,1,0,1,0,0,0,4400,220,5.0,0,81,0,1,0,0,0,0,0,0,0,0
558208,BENE157378,CLM460770,2009-07-09,2009-07-29,PRV51577,2100,PHY338096,None,None,None,0,0.0,None,0.0,0.0,0.0,2.0,None,2,1,1.0,5,200,12,12,1,1,1,0,1,0,1,1,0,0,0,0,0,14240,2810,6.0,0,63,0,1,0,0,0,0,0,0,0,0
558209,BENE158295,CLM306999,2009-04-16,2009-04-16,PRV53083,10,PHY416646,None,None,None,0,0.0,None,0.0,0.0,0.0,1.0,None,2,1,0.0,16,490,12,12,0,1,1,1,1,1,1,1,1,1,0,0,0,640,350,9.0,0,84,0,1,0,0,0,0,0,0,0,0


In [35]:
diag_code = output.iloc[:,45:53]
frequent_itemsets_01 = apriori(diag_code, min_support = 0.01, use_colnames=True)
frequent_itemsets_01

,support,itemsets
0,0.018726,(None_diagcode)
1,0.035655,(V5861_diagcode)
2,0.064026,(2724_diagcode)
3,0.042541,(4011_diagcode)
4,0.031529,(2449_diagcode)
5,0.138014,(4019_diagcode)
6,0.066916,(25000_diagcode)
7,0.036047,(42731_diagcode)
8,0.018796,"(2724_diagcode, 4019_diagcode)"
9,0.018226,"(4019_diagcode, 25000_diagcode)"


In [37]:
res_01 = association_rules(frequent_itemsets_01, metric="confidence", min_threshold=0.01)
res_01

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(2724_diagcode),(4019_diagcode),0.064026,0.138014,0.018796,0.293565,2.127062,0.009959,1.220191
1,(4019_diagcode),(2724_diagcode),0.138014,0.064026,0.018796,0.136187,2.127062,0.009959,1.083538
2,(4019_diagcode),(25000_diagcode),0.138014,0.066916,0.018226,0.132060,1.973525,0.008991,1.075056
3,(25000_diagcode),(4019_diagcode),0.066916,0.138014,0.018226,0.272374,1.973525,0.008991,1.184656


## Chronic Conditions

In [26]:
ChronicCond = ['ChronicCond_Alzheimer','ChronicCond_Heartfailure','ChronicCond_KidneyDisease','ChronicCond_Cancer',\
              'ChronicCond_ObstrPulmonary','ChronicCond_Depression','ChronicCond_IschemicHeart','ChronicCond_Osteoporasis',\
              'ChronicCond_rheumatoidarthritis','ChronicCond_stroke','PotentialFraud']

In [38]:
pd.set_option('display.max_columns', None)
frequent_itemsets_02 = apriori(output[ChronicCond], min_support = 0.4, use_colnames=True)
frequent_itemsets_02

,support,itemsets
0,0.401868,(ChronicCond_Alzheimer)
1,0.590427,(ChronicCond_Heartfailure)
2,0.412002,(ChronicCond_KidneyDisease)
3,0.434807,(ChronicCond_Depression)
4,0.759265,(ChronicCond_IschemicHeart)
5,0.504650,"(ChronicCond_Heartfailure, ChronicCond_Ischemi..."


In [39]:
res_02 = association_rules(frequent_itemsets_02, metric="confidence", min_threshold=0.5)
res_02

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ChronicCond_Heartfailure),(ChronicCond_IschemicHeart),0.590427,0.759265,0.50465,0.854719,1.12572,0.056359,1.657036
1,(ChronicCond_IschemicHeart),(ChronicCond_Heartfailure),0.759265,0.590427,0.50465,0.664656,1.12572,0.056359,1.221350


## Physicians

In [12]:
def phys_freq(df):
    
    # Focus on Physician columns only
    columns = ['AttendingPhysician','OperatingPhysician','OtherPhysician']
        
    new_df = df[columns]
    
    # List of Physician codes and their frequency in that column
    new_dct = []
    for y in new_df:
        new_dct.append(dict(df[y].value_counts(ascending=False,normalize=True)))
    
    # Filter physician that make up more than 0.2% of the physicians
    high_freq = []  
    for i in new_dct:    
        for key,value in i.items():
            if value > 0.002:
                high_freq.append(key)
            
    return high_freq

In [13]:
common_phys = list(set(phys_freq(output)))
common_phys

['PHY338032',
 'PHY341578',
 'None',
 'PHY357120',
 'PHY330576',
 'PHY337425',
 'PHY314027',
 'PHY327046',
 'PHY412132',
 'PHY350277',
 'PHY423534']

In [14]:
del common_phys[2]

In [15]:
common_phys

['PHY338032',
 'PHY341578',
 'PHY357120',
 'PHY330576',
 'PHY337425',
 'PHY314027',
 'PHY327046',
 'PHY412132',
 'PHY350277',
 'PHY423534']

In [16]:
def newdf_PHY_highfreq(df):
    columns = ['AttendingPhysician','OperatingPhysician','OtherPhysician'] 
    new_df = df[columns]
    
    common_phys = ['PHY338032','PHY341578','PHY357120','PHY330576','PHY337425','PHY314027','PHY327046',\
     'PHY412132','PHY350277','PHY423534']
    
    for col in common_phys:
        df[str(col)] = np.where((df[columns].eq(col)).any(1, skipna=True), 1, 0)
        df['None_PHY'] = np.where((df[columns].eq('None')).all(1,skipna=True), 1, 0)
    
    df = df.drop(df[columns],axis=1)
    
    return df

In [17]:
new = newdf_PHY_highfreq(output)
new

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,Inpatient,ClmProcedureCode,ClmProcedureCode_count,ClmDiagnosisCode_count,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,num_chronic,PotentialFraud,Age,V5869_diagcode,other_diagcode,None_diagcode,V5861_diagcode,2724_diagcode,4011_diagcode,2449_diagcode,4019_diagcode,25000_diagcode,42731_diagcode,PHY338032,None_PHY,PHY341578,PHY357120,PHY330576,PHY337425,PHY314027,PHY327046,PHY412132,PHY350277,PHY423534
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,2009-04-12 00:00:00,1,1068.0,2009-04-18 00:00:00,1.0,0.0,0.0,9.0,None,1,1,0.0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,7.0,1,66,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,BENE16973,CLM565430,2009-09-06,2009-09-06,PRV55912,50,None,0,0.0,None,0.0,0.0,0.0,9.0,None,1,1,0.0,39,310,12,12,1,1,1,0,0,0,0,1,0,0,0,24000,2136,450,200,4.0,1,77,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,BENE17521,CLM34721,2009-01-20,2009-02-01,PRV55912,19000,2009-01-20 00:00:00,1,1068.0,2009-02-01 00:00:00,1.0,1.0,2.0,9.0,None,2,1,0.0,39,230,12,12,1,1,1,0,1,0,0,1,0,0,0,19000,1068,100,20,5.0,1,95,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,BENE21718,CLM72336,2009-10-17,2009-11-04,PRV55912,17000,2009-10-17 00:00:00,1,1068.0,2009-11-04 00:00:00,1.0,1.0,1.0,9.0,None,1,1,0.0,39,600,12,12,0,0,0,0,0,0,0,1,1,0,0,17000,1068,1050,540,2.0,1,87,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,BENE22934,CLM73394,2009-10-25,2009-10-29,PRV55912,13000,2009-10-25 00:00:00,1,1068.0,2009-10-29 00:00:00,1.0,1.0,1.0,7.0,None,2,1,0.0,39,280,12,12,0,1,0,0,1,0,1,1,1,1,0,27000,2136,450,160,6.0,1,79,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,BENE154147,CLM394122,2009-06-02,2009-06-04,PRV54050,500,None,0,0.0,None,0.0,0.0,0.0,3.0,None,2,1,0.0,23,400,12,12,0,0,0,0,0,0,0,1,1,0,0,0,0,890,120,2.0,0,84,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
558207,BENE154687,CLM184358,2009-02-08,2009-02-08,PRV54302,3300,None,1,0.0,None,0.0,0.0,0.0,1.0,None,1,2,0.0,25,150,12,12,1,0,1,0,0,0,1,1,0,1,0,0,0,4400,220,5.0,0,81,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
558208,BENE157378,CLM460770,2009-07-09,2009-07-29,PRV51577,2100,None,0,0.0,None,0.0,0.0,0.0,2.0,None,2,1,1.0,5,200,12,12,1,1,1,0,1,0,1,1,0,0,0,0,0,14240,2810,6.0,0,63,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
558209,BENE158295,CLM306999,2009-04-16,2009-04-16,PRV53083,10,None,0,0.0,None,0.0,0.0,0.0,1.0,None,2,1,0.0,16,490,12,12,0,1,1,1,1,1,1,1,1,1,0,0,0,640,350,9.0,0,84,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
phys_column = output.iloc[:,53:]

In [40]:
frequent_itemsets_03 = apriori(phys_column, min_support = 0.001, use_colnames=True)
frequent_itemsets_03

,support,itemsets
0,0.003488,(PHY338032)
1,0.002657,(None_PHY)
2,0.003300,(PHY341578)
3,0.002332,(PHY357120)
4,0.004572,(PHY330576)
5,0.003262,(PHY337425)
6,0.002225,(PHY314027)
7,0.002205,(PHY327046)
8,0.003844,(PHY412132)
9,0.002916,(PHY350277)


In [42]:
res_03 = association_rules(frequent_itemsets_03, metric="confidence", min_threshold=0.01)
res_03

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(PHY338032),(PHY357120),0.003488,0.002332,0.001007,0.288649,123.75358,0.000999,1.402497
1,(PHY357120),(PHY338032),0.002332,0.003488,0.001007,0.431644,123.75358,0.000999,1.753323


# Summary

## Diagnosis Code
- Created new columns to view Diagnoses codes that make up more than 1% of all codes and dropped everything else.
- With a 0.01 minimum support, we found associations between 2724 and 4019, as well as 4019 and 25000.
- Because of the low support and confidence, we decided not to create new features with this association.

## Chronic Conditions
- With a 0.5 minimum support, we found associations between Heartfailure and IschemicHeart. 
- Since this is intuitive information, we decided not to create new features with this association.

## Physicians
- Created new columns to view Physicans that make up more than 0.2% of all physicans and dropped everything else.
- With a 0.001 minimum support, we found associations between PHY388032 and PHY357120.
- Because of the low support and confidence, we decided not to create new features with this association.